# Import Libraries

In [1]:
from swarmintelligence import WhaleOptimizationAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/WOA/optimal_params.pkl', 'rb'))
optimal_params

{'n_whales': 25, 'maxIteration': 100, 'constanta': 5}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
n_runs = 30

In [6]:
woa_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                woa_evaluation_results['image_name'].append(image_name)
                woa_evaluation_results['thresholds'].append(threshold)
                woa_evaluation_results['fitness_function'].append(fitness_function)
                woa_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # WOA optimization
                    woa = WhaleOptimizationAlgorithm(
                        k=threshold,
                        n_whales=optimal_params['n_whales'],
                        maxIteration=optimal_params['maxIteration'],
                        constanta=optimal_params['constanta'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    whales, best_thresholds = woa.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(woa.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(woa.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                woa_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                woa_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                woa_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                woa_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                woa_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                woa_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                woa_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                woa_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                woa_evaluation_results['Regions'].append(mean_regions)
                woa_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                woa_evaluation_results['Fitness'].append(fitness_values)
                woa_evaluation_results['MSE'].append(list_mse)
                woa_evaluation_results['RMSE'].append(list_rmse)
                woa_evaluation_results['PSNR'].append(list_psnr)
                woa_evaluation_results['SSIM'].append(list_ssim)
                woa_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.6717399702634
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.5459555408786
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.5046924348085
1948.7183902321062
1948.7183902321062
1948.3379786574878
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7106129145502
1948.7183902321062
1948.7183902321062
1948.6717399702634
1948.7183902321062
1948.6409800416593
1948.6409800416593
1948.7183902321062
1948.7183902321062
1948.7106129145502
1948.7183902321062
Mean Fitness = 1948.684049578678
	 AirplaneF16.tiff | Threshold =  3
2024.8020017977378
2024.4990114744992
2024.8375098105803
2024.5847077974663
2024.8375098105803
2024.5336604100748
2024.4202040299483
2024.3506995533467
2024.8375098105803
2024.8375098105803
2024.6932725102565
2024.7838782335534
2024.6823418213698
2024.8375098105803
2024.8375098105803
2023.6089088489975
2024.799080138318
202

3208.403706380546
3207.928354754653
Mean Fitness = 3221.1942329186472
	 Mandrill.tiff | Threshold =  2
1548.9859586850848
1549.0271327708897
1548.3075195992974
1548.9952130399997
1549.0271327708897
1549.0271327708897
1548.9952130399997
1549.0271327708897
1549.0271327708897
1548.6645908143018
1548.9952130399997
1549.0271327708897
1548.9859586850848
1549.0271327708897
1548.7861468406588
1548.7861468406588
1548.9952130399997
1548.9859586850848
1548.9952130399997
1549.0271327708897
1548.9859586850848
1549.0271327708897
1548.9952130399997
1548.9952130399997
1548.9952130399997
1548.9952130399997
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
Mean Fitness = 1548.9599294072275
	 Mandrill.tiff | Threshold =  3
1548.9952130399997
1638.9689179665552
1639.338310828638
1639.393273956095
1639.463360998692
1549.0271327708897
1639.151262123955
1638.7760446646016
1639.466171099567
1639.463360998692
1639.321845931115
1639.4006244533603
1639.393273956095
1549.0271327708897
16

4216.992617943262
4216.667472476825
4217.428879472662
4217.602303055319
4214.876456951606
4217.81370197124
Mean Fitness = 4211.060505673095


## Visualize Results using DataFrame

In [7]:
woa_evaluation_results_df =pd.DataFrame(woa_evaluation_results)
woa_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[113, 173]",0.567668,1948.684050,2271.405893,47.659225,14.567872,0.794365,0.940206,"[[113, 255, 255, 255, 255, 255, 255, 255, 255,...","[0.8335554599761963, 0.5229918956756592, 0.644...","[1948.7183902321062, 1948.7183902321062, 1948....","[2270.6405181884766, 2270.6405181884766, 2270....","[47.651238369936166, 47.651238369936166, 47.65...","[14.569319775617668, 14.569319775617668, 14.56...","[0.7944112258870609, 0.7944112258870609, 0.794...","[0.9402201034707982, 0.9402201034707982, 0.940..."
1,AirplaneF16.tiff,3,otsu,max,"[92, 144, 190]",1.237992,2024.679989,1797.729825,42.397602,15.584393,0.782605,0.952866,"[[92, 255, 255, 190, 190, 190, 255, 190, 255, ...","[1.0903682708740234, 0.9572365283966064, 1.183...","[2024.8020017977378, 2024.4990114744992, 2024....","[1779.3615493774414, 1857.3043060302734, 1774....","[42.18247917533346, 43.09645352033359, 42.1229...","[15.62816159301072, 15.441972952527614, 15.640...","[0.7788206538325575, 0.7932986394753002, 0.779...","[0.953067156214341, 0.9520493760049866, 0.9532..."
2,AirplaneF16.tiff,4,otsu,max,"[80, 125, 170, 202]",1.292130,2068.738661,1145.071265,33.823780,17.550243,0.732619,0.968639,"[[80, 202, 202, 202, 202, 202, 202, 202, 202, ...","[1.3603942394256592, 1.3313510417938232, 1.318...","[2068.822052477738, 2069.6676296158644, 2069.4...","[1035.392864227295, 1094.8959617614746, 1100.1...","[32.17752110134177, 33.08921216592312, 33.1687...","[17.979751933043495, 17.737075068785412, 17.71...","[0.7296430777960345, 0.730968542682985, 0.7296...","[0.9708337527631418, 0.9696222212105043, 0.969..."
3,AirplaneF16.tiff,5,otsu,max,"[64, 105, 141, 178, 204]",1.375965,2094.259011,959.695088,30.958897,18.320574,0.744254,0.974599,"[[105, 204, 204, 204, 204, 204, 204, 204, 204,...","[1.309737205505371, 1.3168141841888428, 1.3513...","[2095.923362167883, 2095.6253732048995, 2093.3...","[906.3601837158203, 1022.4945449829102, 914.75...","[30.105816443269237, 31.976468613386785, 30.24...","[18.557795420932415, 18.034193611471036, 18.51...","[0.7457972266893665, 0.7437360879918885, 0.743...","[0.9755629385762955, 0.9733606058142368, 0.975..."
4,Lena.png,2,otsu,max,"[93, 151]",1.246289,1961.795963,2811.014836,53.018571,13.642314,0.612904,0.883503,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.2222926616668701, 1.253821611404419, 1.2581...","[1961.8220125153634, 1961.8220125153634, 1961....","[2813.567070007324, 2813.567070007324, 2813.56...","[53.04306806744237, 53.04306806744237, 53.0430...","[13.638230885124642, 13.638230885124642, 13.63...","[0.6121817460412936, 0.6121817460412936, 0.612...","[0.8833090574580458, 0.8833090574580458, 0.883..."
5,Lena.png,3,otsu,max,"[80, 126, 171]",1.254155,2128.144049,1252.110117,35.384759,17.154580,0.707256,0.934289,"[[171, 171, 171, 171, 171, 171, 171, 171, 171,...","[1.3129560947418213, 1.3703808784484863, 1.371...","[2128.0917920436414, 2128.2352940645033, 2128....","[1243.6808166503906, 1245.9188499450684, 1252....","[35.26585907999961, 35.297575695011524, 35.389...","[17.18371425331339, 17.175906043944984, 17.153...","[0.7056511503344316, 0.706412216330159, 0.7072...","[0.9334739999313126, 0.9341624806213228, 0.934..."
6,Lena.png,4,otsu,max,"[74, 112, 144, 180]",1.360925,2190.857576,796.508380,28.220447,19.120139,0.753190,0.955051,"[[180, 180, 180, 180, 180, 180, 180, 180, 180,...","[1.320828914642334, 1.3714015483856201, 1.2413...","[2184.5304161146537, 2191.680135459974, 2191.8...","[775.5757637023926, 804.8701972961426, 794.494...","[27.849160915589405, 28.370234353916477, 28.18...","[19.23456131702568, 19.073545142185843, 19.129...","[0.7490896957455883, 0.7523448343742074, 0.753...","[0.9584610643742202, 0.9549231481474519, 0.954..."
7,Lena.png,5,otsu,max,"[66, 96, 125, 152, 183]",1.327351,2215.334209,

# Save results

In [8]:
pickle.dump(woa_evaluation_results_df, open('results/evaluation/WOA/WOA_otsu_df.pkl', 'wb'))
pickle.dump(woa_evaluation_results, open('results/evaluation/WOA/WOA_otsu_dict.pkl', 'wb'))